In [1]:
USE_FULL_1900_DIM_MODEL = False # if True use 1900 dimensional model, else use 64 dimensional one.

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set seeds
tf.set_random_seed(42)
np.random.seed(42)

if USE_FULL_1900_DIM_MODEL:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/1900_weights/ 1900_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler1900 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./1900_weights"
    
else:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler64 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./64_weights"

In [3]:
batch_size = 50
b = babbler(batch_size=batch_size, model_path=MODEL_WEIGHT_PATH)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# Before you can train your model, 
sequences = []
with open("lenzi_neg_seqs_3.txt", "r") as source:
    with open("formatted.txt", "w") as destination:
        for i,seq in enumerate(source):
            seq = seq.strip()
            sequences.append(seq)
            if b.is_valid_seq(seq) and len(seq) < 275: 
                formatted = ",".join(map(str,b.format_seq(seq)))
                destination.write(formatted)
                destination.write('\n')

In [5]:
## 
average_hidden_list = []
final_hidden_list = []
hs_list = []
final_cell_list = []


num2 = range(0, 50)
x = 0
y = 50
for i in num2:
    num1 = range(x, y)
    for j in num1:
        avg_hidden, final_hidden, final_cell, hs_out = (b.get_rep_hs(sequences[j]))
        average_hidden_list.append(avg_hidden)
        final_hidden_list.append(final_hidden)
        final_cell_list.append(final_cell)
        hs_list.append(hs_out)
    print('rep')
    x = x + 50
    y = y + 50
    


rep
rep
rep
rep
rep
rep
rep
rep
rep
rep


IndexError: list index out of range

In [6]:
average_hidden_pd = pd.DataFrame(np.row_stack(average_hidden_list))
final_hidden_pd = pd.DataFrame(np.row_stack(final_hidden_list))
hidden_state = pd.DataFrame(np.row_stack(hs_list))
print(hidden_state)

             0         1         2         3         4         5         6   \
0     -0.165004  0.367741  0.358379 -0.652705  0.310745  0.112354  0.319343   
1     -0.117507  0.015849 -0.005665 -0.945347 -0.161479 -0.059104  0.105382   
2     -0.149950  0.035742 -0.040070 -0.982778 -0.080040 -0.096058  0.044528   
3     -0.071795  0.100204 -0.063417 -0.986795  0.008397 -0.125489  0.072548   
4     -0.059268  0.068293 -0.067980 -0.980529 -0.035562 -0.091877  0.018135   
5     -0.006058  0.080899 -0.091961 -0.981459 -0.033209 -0.121571  0.013050   
6     -0.051401  0.110002 -0.107895 -0.978658 -0.035257 -0.102357  0.035238   
7      0.019964  0.069578 -0.095498 -0.978140 -0.050059 -0.092941  0.004678   
8      0.035149  0.082405 -0.081118 -0.986739 -0.028930 -0.133066  0.020592   
9     -0.007347  0.143629 -0.072519 -0.988234 -0.009851 -0.146884  0.055827   
10    -0.011108  0.146124 -0.072513 -0.981836 -0.003423 -0.178665  0.021780   
11     0.102804  0.121156 -0.056406 -0.980689 -0.007

In [7]:
average_hidden_pd.to_csv("lenzi_neg_reps_3.csv")
final_hidden_pd.to_csv("lenzi_neg_finalhidden_3.csv")

In [8]:
import pickle
save_loc = "C:\\Users\\pkinn\\Documents\\UniRep\\full representations\\emi larger set\\"
data_name = 'lenzi_neg_reps_3'
file_append = '.pickle'


fn = save_loc + data_name + 'avg_hidden' + file_append
with open(fn, 'wb') as f:
    pickle.dump(average_hidden_list, f)

fn = save_loc + data_name + 'final_hidden' + file_append
with open(fn, 'wb') as f:
    pickle.dump(final_hidden_list, f)

fn = save_loc + data_name + 'final_cell' + file_append
with open(fn, 'wb') as f:
    pickle.dump(final_cell_list, f)
   
fn = save_loc + data_name + 'hidden_state' + file_append
with open(fn, 'wb') as f:
    pickle.dump(hs_list, f)
   
fn = save_loc + data_name + 'all_output_hs' + file_append
with open(fn, 'wb') as f:
    pickle.dump([average_hidden_list, final_hidden_list, final_cell_list, hs_list], f)